In [5]:
import awswrangler as wr
import pandas as pd
# import pymysql
# import datetime

# Lambda 핸들러 함수. 이벤트와 컨텍스트 매개변수를 받습니다.


# 첫 번째 SQL 쿼리 실행
sql_query = """
    with ad_banner as (SELECT id as bannerid, title, description, pc_img_url, mobile_img_url FROM "moyo_data"."ad_banner"),
    
    ad_schedule as (SELECT distinct title as campaign, start_date, end_date FROM "moyo_data"."ad_schedule"),
    
    banner_stats as (select date_ymd, bannerid, campaign,
            case when bannerid = '220' then 'ad_skt_event_230601' else banner_campaign end as banner_campaign, view_type,
            count(case when category = 'impression' then uuid end) as view_count,
            count(case when category = 'click' then uuid end) as click_count
    from
    (SELECT date(date_ymd) as date_ymd,
            object_id as bannerid,
            trim(json_extract_scalar(event_properties,'$.campaign')) as campaign,
            navigation as view_type,
            uuid,
            category,
            trim(json_extract_scalar(event_properties,'$.banner_campaign')) as banner_campaign
    FROM "moyo_logs"."user_behavior_logs" 
    where date(date_ymd) >= date_add('day',-5,date(current_timestamp at time zone 'asia/seoul')) and object_type = 'banner')
    group by 1, 2, 3, 4, 5)
    
    select a.bannerid, a.campaign, banner_campaign, a.view_type, start_date, end_date, 
            title, description, pc_img_url, mobile_img_url, view_count, click_count, date_ymd
    from banner_stats a
    left outer join ad_banner b
    on a.bannerid = cast(b.bannerid as varchar)
    left outer join ad_schedule c
    on a.campaign = c.campaign
     where date_ymd <> date(current_timestamp at time zone 'asia/seoul') and a.banner_campaign is not null -- 이거 지우고 자시 돌려야 한다 





    """

df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")

bucket = 'moyo-mart'
table_name = 'ad_banner_stats'

res = wr.s3.to_parquet(
    df=df,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions",
    partition_cols=['date_ymd']
)
# 주석 추가: 첫 번째 SQL 쿼리 실행 결과를 Parquet 형식으로 S3에 저장

######################################################################################################

# # 두 번째 SQL 쿼리 실행
# sql_query2 = """
# select * 
# from moyo_logs.user_behavior_logs 
# where 1=1 
#     and date_ymd = '2023-04-01'
# limit 10 
# """
# df = wr.athena.read_sql_query(sql=sql_query2, database="moyo_logs")
# bucket = 'moyo-mart'
# table_name = 'just_test_temp2'
# res = wr.s3.to_parquet(
#     df=df,
#     path=f"s3://{bucket}/{table_name}/",
#     dataset=True,
#     database="moyo_mart",
#     table=table_name,
#     mode="overwrite_partitions",
#     # partition_cols=['date_ym']
# )

# # 주석 추가: 두 번째 SQL 쿼리 실행 결과를 Parquet 형식으로 S3에 저장


/Users/ringokwon/anaconda3/lib/python3.10/site-packages/awswrangler/s3/_write_dataset.py:93: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):
